二维卷积
| |天气1_mean |天气1_max |天气1_min |天气1_std |天气2_mean |... |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|样本1 | | | | | | |
|样本2 | | | | | | |
|样本3 | | | | | | |
|样本4 | | | | | | |
|样本5 | | | | | | |
|样本6 | | | | | | |
|样本7 | | | | | | |

样本1 - 7 为连续的7天的数据。 将第7天的数据作为目标值进行回归

1. 特征的排列顺序可能会有影响
2. 将17-19三年作为训练； 20-21的各一半作为预测

In [1]:
import os
import copy
import random
import numpy as np
import pandas as pd
from PIL import Image

# torch
import torch
import torch.nn as nn
from torch.nn.modules import Conv2d
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

In [2]:
# 使用ACS做二维卷积
data_path = '../data'
count_path = os.path.join(
    data_path, 'counts_ACS.csv'
)
ACS_path = os.path.join(
    data_path, 'newset.csv'
)
df_features = pd.read_csv(ACS_path)  # 使用-3， -6所有数据，扔掉七天和三天，只用当天
df_counts = pd.read_csv(count_path)
df_features['time'] = pd.to_datetime(df_features['time'])
df_counts['time'] = pd.to_datetime(df_counts['time'])
table = pd.merge(left=df_features, right=df_counts, on = 'time')
table['counts'] = table['counts'].replace(np.nan, table['counts'].median())
table['counts'] = table['counts'].replace(-1, table['counts'].median())
table.shape

(1536, 50)

In [3]:
table[table.isnull().values == True].drop_duplicates()

,time,AQI_0,AQI_6,AQI_12,AQI_18,PM2_0,PM2_6,PM2_12,PM2_18,PM10_0,...,pressure_18,direction_0,direction_6,direction_12,direction_18,speed_0,speed_6,speed_12,speed_18,counts
86,2017-03-21,53.500000,63.833333,52.200000,NaN,37.833333,43.500000,23.000000,NaN,48.166667,...,10179.0,280.0,340.0,350.0,290.0,35.0,45.0,20.0,30.0,6.0
196,2017-07-09,43.333333,46.500000,NaN,40.250000,26.166667,27.500000,NaN,12.000000,42.500000,...,9976.0,170.0,180.0,180.0,160.0,60.0,60.0,45.0,35.0,2.0
241,2017-08-23,NaN,NaN,41.200000,39.833333,NaN,NaN,28.000000,20.166667,NaN,...,10040.5,245.0,235.0,220.0,210.0,40.0,30.0,20.0,25.0,3.0
257,2017-09-08,23.666667,16.000000,NaN,NaN,9.333333,8.666667,NaN,NaN,23.500000,...,10153.0,340.0,190.0,350.0,325.0,30.0,40.0,15.0,10.0,5.0
258,2017-09-09,NaN,NaN,21.000000,63.000000,NaN,NaN,8.000000,33.333333,NaN,...,10144.5,260.0,330.0,280.0,200.0,20.0,15.0,10.0,15.0,2.0
349,2017-12-09,94.666667,NaN,NaN,NaN,70.333333,NaN,NaN,NaN,94.666667,...,10102.5,80.0,90.0,215.0,270.0,17.5,45.0,25.0,30.0,7.0
575,2018-07-23,84.666667,59.500000,NaN,62.800000,59.000000,42.833333,NaN,33.000000,118.166667,...,10056.5,220.0,245.0,190.0,180.0,30.0,25.0,15.0,12.5,8.0
607,2018-08-24,29.500000,37.200000,34.250000,NaN,13.250000,12.200000,13.750000,NaN,29.500000,...,10048.5,70.0,30.0,40.0,55.0,35.0,25.0,55.0,50.0,6.0
608,2018-08-25,NaN,15.800000,13.833333,20.500000,NaN,6.400000,7.166667,12.333333,NaN,...,10046.5,55.0,90.0,110.0,65.0,55.0,40.0,35.0,30.0,2.0
609,2018-08-26,23.500000,NaN,NaN,NaN,12.000000,NaN,NaN,NaN,23.500000,...,10111.0,170.0,350.0,225.0,170.0,15.0,15.0,10.0,15.0,12.0


In [4]:
table = table.fillna(method='ffill', axis = 0)
table.to_csv(os.path.join(data_path, "mergedNewAcs.csv"), index=False)

# 阈值处理
详细见阈值处理.ipynb

In [5]:
table['AQI_0'] = table['AQI_0'].clip(upper = 400)
table['AQI_6'] = table['AQI_6'].clip(upper = 400)
table['AQI_12'] = table['AQI_12'].clip(upper = 300)
table['AQI_18'] = table['AQI_18'].clip(upper = 400)

table['PM2_0'] = table['PM2_0'].clip(upper = 300)
table['PM2_6'] = table['PM2_6'].clip(upper = 300)
table['PM2_12'] = table['PM2_12'].clip(upper = 250)
table['PM2_18'] = table['PM2_18'].clip(upper = 300)

table['PM10_0'] = table['PM10_0'].clip(upper = 400)
table['PM10_6'] = table['PM10_6'].clip(upper = 400)
table['PM10_12'] = table['PM10_12'].clip(upper = 300)
table['PM10_18'] = table['PM10_18'].clip(upper = 400)

table['SO2_0'] = table['SO2_0'].clip(upper = 100)
table['SO2_6'] = table['SO2_6'].clip(upper = 100)
table['SO2_12'] = table['SO2_12'].clip(upper = 100)
table['SO2_18'] = table['SO2_18'].clip(upper = 100)

table['NO2_0'] = table['NO2_0'].clip(upper = 120)
table['NO2_6'] = table['NO2_6'].clip(upper = 120)
table['NO2_12'] = table['NO2_12'].clip(upper = 120)
table['NO2_18'] = table['NO2_18'].clip(upper = 120)

# O3 四个时段显示出明显差异，不clip

table['CO_0'] = table['CO_0'].clip(upper = 4)
table['CO_6'] = table['CO_6'].clip(upper = 4)
table['CO_12'] = table['CO_12'].clip(upper = 4)
table['CO_18'] = table['CO_18'].clip(upper = 4)


table['pressure_0'] = table['pressure_0'].clip(lower = 9950)
table['pressure_6'] = table['pressure_6'].clip(lower = 9950)
table['pressure_12'] = table['pressure_12'].clip(lower = 9950)
table['pressure_18'] = table['pressure_18'].clip(lower = 9950)

In [6]:
table.iloc[728:732, 1:29] = np.nan
table = table.fillna(table.mean())
table.shape

C:\Users\jin12\AppData\Local\Temp\ipykernel_9320\1596390325.py:2: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  table = table.fillna(table.mean())


(1536, 50)

In [7]:
for row in table.columns[1:-1]:
    table.loc[:, row] = (table.loc[:, row] - table.loc[:, row].mean()) / table.loc[:, row].std()
table['counts'].describe()


count    1536.000000
mean        8.338542
std         4.400471
min         0.000000
25%         5.000000
50%         8.000000
75%        11.000000
max        28.000000
Name: counts, dtype: float64

In [8]:

def filter(x):
    if x<=5:
        return 0
    elif x>=11:
        return 1
    else:
        return 2
table['counts'] = table.counts.apply(lambda x:filter(x))
table = table[(table['counts'] == 0) | (table['counts'] == 1)]

In [9]:
def to_img(df):
    # 2016/12/25 -- 2016-12/31 的标签记为 2017/01/01的counts
    imgs = []
    labels = []
    for col in range(df.shape[0] - 7):
        temp_df = df.iloc[col:col+8]
        img = temp_df.iloc[0:7, 1:-1]
        img = img.values
        lable = temp_df.iloc[7, -1]
        imgs.append(img)
        labels.append(lable)
    return imgs, labels
train_ratio = 0.7
fir, sec = to_img(table)
data =[[], []]
data[0].extend(fir[:1096])
data[0].extend(fir[1338:])
data[1].extend(sec[:1096])
data[1].extend(sec[1338:])

datasets = {
    'train': [[], []],
    'val': [[], []]
}

for i in range(len(data[0])):
    if random.uniform(0, 1) < train_ratio:
        datasets['train'][0].append(data[0][i])
        datasets['train'][1].append(data[1][i])
    else:
        datasets['val'][0].append(data[0][i])
        datasets['val'][1].append(data[1][i])


In [10]:
class myData(Dataset):
    def __init__(self, datasets, mode):
        super(myData, self).__init__()
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((224, 224)),
        ])
        
        self.imgs, self.labels = datasets[mode][0], datasets[mode][1]
    
    def __getitem__(self, idx):
        sample = Image.fromarray(self.imgs[idx])
        sample = sample.convert('RGB')
        sample = self.transform(sample)
        return sample, self.labels[idx]
    
    def __len__(self):
        return len(self.imgs)

img_datasets = {
    x:myData(datasets, x) for x in ['train', 'val']
}

dataset_sizes = {
    x: len(img_datasets[x]) for x in ['train', 'val']
}

train_loader = DataLoader(
    img_datasets['train'],
    batch_size=1,
    shuffle=True,
)

val_loader = DataLoader(
    img_datasets['val'],
    batch_size=1,
    shuffle=False
)

dataloaders = {
    'train': train_loader,
    'val': val_loader
}

In [11]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def train_model(model, criterion, optimizer, schedule ,epoches = 75):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(epoches):
        print("in epoch: %d" % epoch)
        count = 0
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_correct = 0.0
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to("cuda:0")
                labels = labels.to("cuda:0")

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    # inputs = inputs.type(torch.FloatTensor).to("cuda:0")
                    # print(labels)
                    outputs = model(inputs)
                    # print(outputs)
                    _, idx = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_correct += torch.sum(idx == labels.data)
            if phase == 'train':
                schedule.step()
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_correct.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format( 
                phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:  
                best_acc = epoch_acc                                
                best_model_wts = copy.deepcopy(model.state_dict()) 
    print('Best val Acc: {:4f}'.format(best_acc))    

    model.load_state_dict(best_model_wts)   
    return model

In [12]:
class myModel(nn.Module):
    def __init__(self):
        super(myModel, self).__init__()

        # 32*32
        self.stage1 = nn.Sequential(
            # 将特征扩充
            nn.Conv2d(1, 32, 4, padding = 1, bias=True),
            nn.ReLU(True),
            nn.MaxPool2d(3, 2, padding = 1),
        )
        # 16*16
        self.stage2 = nn.Sequential(
            nn.Conv2d(32, 64, 2, stride=2,padding=0,bias=True),
            nn.ReLU(True),
            nn.MaxPool2d(3, 2, padding = 1),
        )

        # 4*4  *  64
        self.fc = nn.Sequential(
            nn.Linear(4*4*64, 4)
        )

    def forward(self, x):
        out = self.stage1(x)
        # print("1 out:",out.shape)
        out = self.stage2(out)
        # print("2 out:",out.shape)
        out = out.view(1, -1)
        # print("view out:",out.shape)
        out = self.fc(out)
        # print(out)
        return out

In [13]:
# model = myModel()
# # model = model.to("cuda:0")

# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr = 0.05, momentum=0.1)
# lr_sche = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.25)

# model = train_model(model, criterion, optimizer, lr_sche, 100)

In [14]:
from torchvision import models
model_res = models.resnet18(pretrained=True)
num_ftrs = model_res.fc.in_features
model_res.fc = nn.Linear(num_ftrs, 2)
model_res = model_res.to("cuda:0")

criterion = nn.CrossEntropyLoss()
optimizer_res = torch.optim.SGD(model_res.parameters(), lr = 0.02, momentum=0.1)
lr_sche_res = torch.optim.lr_scheduler.StepLR(optimizer_res, step_size=10, gamma=0.1)

model_res = train_model(model_res, criterion, optimizer_res, lr_sche_res, epoches=30)

in epoch: 0
train Loss: 1.6508 Acc: 0.4554
val Loss: 1.9079 Acc: 0.5125
in epoch: 1
train Loss: 1.0541 Acc: 0.5089
val Loss: 1.3977 Acc: 0.4667
in epoch: 2
train Loss: 0.9768 Acc: 0.5018
val Loss: 1.0686 Acc: 0.5125
in epoch: 3
train Loss: 0.9466 Acc: 0.4839
val Loss: 1.4215 Acc: 0.5125
in epoch: 4
train Loss: 0.8916 Acc: 0.4696
val Loss: 1.5564 Acc: 0.5750
in epoch: 5
train Loss: 0.8756 Acc: 0.5214
val Loss: 9.5274 Acc: 0.5125
in epoch: 6
train Loss: 0.8853 Acc: 0.4875
val Loss: 2.9874 Acc: 0.5125
in epoch: 7
train Loss: 0.8468 Acc: 0.5000
val Loss: 1.2560 Acc: 0.5125
in epoch: 8
train Loss: 0.8338 Acc: 0.5268
val Loss: 2.0523 Acc: 0.5167
in epoch: 9
train Loss: 0.8415 Acc: 0.5000
val Loss: 5.2685 Acc: 0.5125
in epoch: 10
train Loss: 0.7092 Acc: 0.5393
val Loss: 4.3977 Acc: 0.5583
in epoch: 11
train Loss: 0.6942 Acc: 0.5589
val Loss: 1.2153 Acc: 0.5500
in epoch: 12
train Loss: 0.6869 Acc: 0.5500
val Loss: 0.9059 Acc: 0.5542
in epoch: 13
train Loss: 0.6897 Acc: 0.5536
val Loss: 1.3204 

In [15]:
# 保存第一次训练的最佳结果
def vali(M ,dataset):
    M.eval()
    with torch.no_grad():
        correct = 0
        for (data, target) in val_loader:
            data, target = data.to("cuda:0"), target.to("cuda:0")

            pred = M(data)
            _, id = torch.max(pred, 1)
            correct += torch.sum(id == target.data)
        print("test accu: %.03f%%" % (100 * correct / len(dataset)))
    return (100 * correct / len(dataset)).item()
test_accu = int(vali(model_res, img_datasets['val']) * 100)
model_name = 'val_{}.pkl'.format(test_accu)

# torch.save(model_res.state_dict(), os.path.join("../myModel", model_name))

test accu: 59.167%


In [16]:
# model_ft = models.resnet18(pretrained=False)
# num_ftrs = model_ft.fc.in_features            
# model_ft.fc = nn.Linear(num_ftrs, 2)  
# model_ft.load_state_dict(torch.load(os.path.join("../myModel", model_name)))

# for param in model_ft.parameters():
#     param.requires_grad = False
    
# num_ftrs = model_ft.fc.in_features            
# model_ft.fc = nn.Linear(num_ftrs, 2)           
# model_ft = model_ft.to("cuda:0")

# criterion = nn.CrossEntropyLoss()       

# optimizer_ft = torch.optim.SGD(model_ft.fc.parameters(), lr=0.001, momentum=0.9)   

# exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1) 

# model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,epoches=15)    # 模型训练

In [17]:
# test_accu = int(vali(model_ft, img_datasets['val']) * 100)
# model_name = 'val_{}.pkl'.format(test_accu)

# torch.save(model_ft.state_dict(), os.path.join("../myModel", model_name))